In [128]:
import pandas as pd

In [129]:
data = pd.DataFrame({'Прогноз погоды':['Пасмурно', 'Солнечно', 'Солнечно', 'Пасмурно', 'Пасмурно', 'Дождливо', 'Солнечно', 'Дождливо', 'Дождливо', 'Пасмурно'],
                     'Влажность%':[87,80,80,75,75,80,80,80,85,87], 'Ветер':[False,True,True,True,True,False,True,False,False,False], 
                     'Играть ли в гольф':[1,1,1,1,1,0,0,0,0,1]})

In [130]:
data

,Прогноз погоды,Влажность%,Ветер,Играть ли в гольф
0,Пасмурно,87,False,1
1,Солнечно,80,True,1
2,Солнечно,80,True,1
3,Пасмурно,75,True,1
4,Пасмурно,75,True,1
5,Дождливо,80,False,0
6,Солнечно,80,True,0
7,Дождливо,80,False,0
8,Дождливо,85,False,0
9,Пасмурно,87,False,1


In [131]:
X = data[['Прогноз погоды', 'Влажность%', 'Ветер']]
X_true = data[['Прогноз погоды', 'Влажность%', 'Ветер']][data['Играть ли в гольф'] == 1]

all_indexes = set(data.index)
X_true_indexes = set(X_true.index)

In [132]:
def get_elementary_subsets(X): #get elementary subsets from set of objects
    subsets = []
    for i in range(len(X)):
        similar = [i]
        for j in range(i + 1, len(X)):
            if X.iloc[i].equals(X.iloc[j]):
                similar.append(j)

        if not any(set(similar) & set(sub) for sub in subsets):
            subsets.append(similar)

    return subsets

In [133]:
def get_lower(elementary, X_true_indexes): #get a lower approximation
    lower_init = [elem for elem in elementary if set(elem).issubset(X_true_indexes)]
    lower = [elem[0] for elem in lower_init]
    all_lower = [j for elem in lower_init for j in elem]

    return lower_init, lower, all_lower

In [134]:
def get_upper(elementary, X_true_indexes): #get an upper approximation
    upper_init = [elem for elem in elementary if set(elem).intersection(X_true_indexes) != set()]
    upper = [elem[0] for elem in upper_init]
    all_upper = [j for elem in upper_init for j in elem]
    
    return upper_init, upper, all_upper

In [135]:
def get_pos_rule(pos_dataframe): #get a rule to print only positive objects
    res = 'если '
    for item in pos_dataframe.items():
        if item[0] != 'Играть ли в гольф':
            res += f'({item[0]}='
            value = ''
            for l in range(len(item[1])):
                if not value:
                    value = str(item[1][l])
                elif str(item[1][l]) in value:
                    pass
                else:
                    value += str(f'V{item[1][l]}')
            res += value
            res += ')&'  
        else:
            res = res[:-1]
            res += ', то играть ли=играть'
    return res

In [136]:
def get_maybe_rule(maybe_dataframe): #get a rule to print objects are able to be positive
    res = 'если '
    for item in maybe_dataframe.items():
        if item[0] != 'Играть ли в гольф':
            res += f'({item[0]}='
            value = ''
            for l in range(len(item[1])):
                if not value:
                    value = str(item[1][l])
                elif str(item[1][l]) in value:
                    pass
                else:
                    value += str(f'V{item[1][l]}')
            res += value
            res += ')&'  
        else:
            res = res[:-1]
            res += ', то играть ли=возможно играть'
    return res

In [137]:
def get_neg_rule(not_pos_dataframe): #get a rule to print only negative objects
    res = 'если '
    for item in not_pos_dataframe.items():
        if item[0] != 'Играть ли в гольф':
            res += f'({item[0]}='
            value = ''
            for l in range(len(item[1])):
                if not value:
                    value = str(item[1][l])
                elif str(item[1][l]) in value:
                    pass
                else:
                    value += str(f'V{item[1][l]}')
            res += value
            res += ')&'  
        else:
            res = res[:-1]
            res += ', то играть ли=не играть'
    return res

In [138]:
elementary = get_elementary_subsets(X)

In [139]:
lower_init, lower, all_lower = get_lower(elementary, X_true_indexes)
upper_init, upper, all_upper = get_upper(elementary, X_true_indexes) 

In [140]:
approximation_rate = len(all_lower)/len(all_upper) #get an approximation rate

In [141]:
POS_x = lower
BOND_x = set(upper).difference(set(lower))
not_POS_x = all_indexes.difference(set(all_upper))  

In [142]:
not_pos_dataframe = pd.DataFrame([data.loc[el] for el in not_POS_x], index=range(len(not_POS_x)))
pos_dataframe = pd.DataFrame([data.loc[el] for el in POS_x], index=range(len(POS_x)))
maybe_dataframe = pd.DataFrame([data.loc[el] for el in BOND_x], index=range(len(BOND_x)))

In [143]:
pos_rule = get_pos_rule(pos_dataframe)
neg_rule = get_neg_rule(not_pos_dataframe)
maybe_rule = get_maybe_rule(maybe_dataframe)

In [144]:
#to print all the rules we get and the approximation rate
print(f'1) {pos_rule}')
print(f'2) {neg_rule}')
print(f'3) {maybe_rule}')
print(f'approximation rate is {approximation_rate}')

1) если (Прогноз погоды=Пасмурно)&(Влажность%=87V75)&(Ветер=FalseVTrue), то играть ли=играть
2) если (Прогноз погоды=Дождливо)&(Влажность%=85V80)&(Ветер=False), то играть ли=не играть
3) если (Прогноз погоды=Солнечно)&(Влажность%=80)&(Ветер=True), то играть ли=возможно играть
approximation rate is 0.5714285714285714
